1. [Cross validation](#cross-validation)
2. [Побдбор гиперпараметров](#побдбор-гиперпараметров)
3. [Sklearn princips](#sklearn-princips)

In [2]:
import pandas as pd

df = pd.read_csv('../3_Classification/Churn_Modelling.csv')# read the csv file

df.head()# show the first 5 rows of the dataframe

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
from sklearn.model_selection import train_test_split

# split the data into train, validation and test sets
train, test = train_test_split(df, train_size=0.6, random_state=42, stratify=df['Exited'])
val, test = train_test_split(test, train_size=0.5, random_state=42, stratify=test['Exited'])


In [4]:
# ! pip install catboost

In [20]:
from catboost import CatBoostClassifier


X = ['CustomerId', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

cat_features = ['Geography','Gender']

y = ['Exited']

In [21]:

from catboost import Pool

train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                 )

valid_data = Pool(data=val[X],
                  label=val[y],
                  cat_features=cat_features
                 )

test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                 )


In [22]:

params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [23]:
model = CatBoostClassifier(**params)

In [31]:
model.fit(train_data,eval_set=valid_data)


0:	test: 0.8338545	best: 0.8338545 (0)	total: 3.35ms	remaining: 2.3s
100:	test: 0.8725090	best: 0.8725090 (100)	total: 243ms	remaining: 1.41s
200:	test: 0.8784656	best: 0.8784656 (200)	total: 486ms	remaining: 1.17s
300:	test: 0.8812676	best: 0.8812676 (300)	total: 729ms	remaining: 932ms
400:	test: 0.8823868	best: 0.8824376 (390)	total: 971ms	remaining: 690ms
500:	test: 0.8834491	best: 0.8834522 (498)	total: 1.22s	remaining: 450ms
600:	test: 0.8834737	best: 0.8836554 (564)	total: 1.47s	remaining: 208ms
685:	test: 0.8838694	best: 0.8838694 (685)	total: 1.68s	remaining: 0us

bestTest = 0.8838694083
bestIteration = 685



In [35]:
n_iters = model.best_iteration_ + 1 # get the best number of iterations
n_iters

686

In [36]:
# retrain the model on the full training data
params = {'iterations':n_iters,
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}
     

model = CatBoostClassifier(**params) # create a new model with the best number of iterations
     

train_full = pd.concat([train,val]) # combine the train and validation data
     

# create a Pool object for the full training data
train_full_data = Pool(train_full[X],
                       label=train_full[y],
                       cat_features=cat_features)
     

model.fit(train_full_data) # fit the model on the full training data

0:	total: 5.37ms	remaining: 3.68s
100:	total: 346ms	remaining: 2.01s
200:	total: 616ms	remaining: 1.49s
300:	total: 899ms	remaining: 1.15s
400:	total: 1.18s	remaining: 841ms
500:	total: 1.46s	remaining: 539ms
600:	total: 1.74s	remaining: 246ms
685:	total: 1.97s	remaining: 0us


In [28]:
from sklearn.metrics import roc_auc_score
test['y_score_no_cross_val'] = model.predict_proba(test_data)[:,1] # predict the probabilities on the test set
# calculate the AUC score on the test set     
roc_auc_score(test['Exited'],test['y_score_no_cross_val'])
     

0.8735839074822127

# Cross validation
https://github.com/catboost/tutorials/blob/master/classification/classification_tutorial.ipynb

In [38]:
from catboost import cv


In [41]:
params = {
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [43]:
# так будет обучено пять моделей 
cv_data = cv(
    params=params,  # параметры модели
    pool=train_full_data,  # данные для обучения
    fold_count=5,  # количество фолдов (разбиений) для кросс-валидации
    shuffle=True,  # перемешивать данные перед разбиением
    partition_random_seed=42,  # случайное зерно для разбиения данных
    stratified=False,  # использовать стратифицированное разбиение, что бы не показывать каждую итерацию
    verbose=False  # выводить подробную информацию о процессе
    # plot=True  # построить график (если требуется)
)

Training on fold [0/5]

bestTest = 0.866360207
bestIteration = 881

Training on fold [1/5]

bestTest = 0.8716621864
bestIteration = 998

Training on fold [2/5]

bestTest = 0.8695412245
bestIteration = 540

Training on fold [3/5]

bestTest = 0.8795945701
bestIteration = 942

Training on fold [4/5]

bestTest = 0.8586794872
bestIteration = 750



In [46]:
# хранится информация о каждой итерации и рассчитана средняя метрика на тесте и трейне
cv_data

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.818479,0.009480,0.685020,0.000209,0.684939,0.000262
1,1,0.823848,0.010334,0.677216,0.000374,0.677055,0.000503
2,2,0.831819,0.009265,0.668989,0.000472,0.668799,0.000751
3,3,0.834559,0.008771,0.662008,0.001326,0.661753,0.000689
4,4,0.834483,0.008218,0.654599,0.001429,0.654270,0.000898
...,...,...,...,...,...,...,...
995,995,0.868703,0.007809,0.329986,0.016873,0.284097,0.003644
996,996,0.868715,0.007781,0.329972,0.016875,0.284057,0.003631
997,997,0.868718,0.007804,0.329972,0.016876,0.284003,0.003636
998,998,0.868725,0.007777,0.329969,0.016855,0.283943,0.003613


In [47]:
cv_data['test-AUC-mean'].max() # максимальное значение AUC на тесте

0.8688218559102292

In [48]:
cv_data['test-AUC-mean'].idxmax() # номер итерации с максимальным значением AUC на тесте

741

In [49]:
cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()] # информация о итерации с максимальным значением AUC на тесте

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
741,741,0.868822,0.007457,0.330818,0.017005,0.297428,0.004326


In [53]:
n_iters = cv_data['test-AUC-mean'].idxmax()  # получаем лучшее количество итераций
n_iters

741

In [54]:
n_iters = cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]['iterations'].values[0] # получаем лучшее количество итераций
n_iters

741

    когда получили необходимое количество итераций для модели, для лучшего результата, обучаем заново с количеством итераций

In [55]:
params = {
            'iterations':n_iters,# получаем лучшее количество итераций
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [56]:
model = CatBoostClassifier(**params) # создаем новую модель с лучшими параметрами

In [57]:
model.fit(train_full_data) # обучаем модель на полных данных

0:	total: 3.33ms	remaining: 2.46s
100:	total: 308ms	remaining: 1.95s
200:	total: 586ms	remaining: 1.57s
300:	total: 871ms	remaining: 1.27s
400:	total: 1.17s	remaining: 989ms
500:	total: 1.45s	remaining: 694ms
600:	total: 1.81s	remaining: 422ms
700:	total: 2.21s	remaining: 126ms
740:	total: 2.35s	remaining: 0us


In [58]:
test['y_score_cross_val'] = model.predict_proba(test_data)[:,1] # предсказываем вероятности на тесте

In [59]:
from sklearn.metrics import roc_auc_score


In [61]:
roc_auc_score(test['Exited'],test['y_score_no_cross_val'])


0.8735839074822127

In [60]:
roc_auc_score(test['Exited'],test['y_score_cross_val']) # считаем AUC на тесте

0.8737196364315009

    здесь особо улучшения нет. но более уверенный подход. и в меньшей степени зависим от рандома

# Побдбор гиперпараметров
    кетбуст прекрасно работает без предворительной настройки.
    но можно и самому перебрать какие то параметры. можно посмотреть все параметры и перебирать различныые комбинации и искать 
    комбинацию где будет лучшее качество. и для этого есть спец функция grid_search
https://github.com/catboost/catboost/blob/master/catboost/tutorials/events/2020_06_04_catboost_tutorial/catboost_features.ipynb

https://youtu.be/ZaP5qFSIcIw?t=2043

https://github.com/catboost/catboost/blob/master/catboost/tutorials/hyperparameters_tuning/hyperparameters_tuning.ipynb

In [62]:
model.get_all_params() # получаем все параметры модели

{'nan_mode': 'Min',
 'eval_metric': 'AUC',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 741,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'random_score_type': 'NormalWithModelSizeDecrease',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=

In [63]:
# это параметры которые изменять не будем
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42}

In [64]:
model = CatBoostClassifier(**params) # создаем модель

In [67]:
# сама сетка параметров. буду перебирать два значения learning_rate и два значения depth. Всего 4 комбинации
# будет построено 4 модели и найдется наилучшее качество
grid = {'learning_rate': [0.01, 0.1],
        'depth': [5, 6]}

In [68]:
result = model.grid_search(grid, train_full_data, verbose=False)


0:	test: 0.7811641	best: 0.7811641 (0)	total: 18.3ms	remaining: 18.3s
100:	test: 0.8435462	best: 0.8435769 (99)	total: 626ms	remaining: 5.58s
200:	test: 0.8508923	best: 0.8508923 (200)	total: 1.15s	remaining: 4.56s
300:	test: 0.8547897	best: 0.8547897 (300)	total: 1.63s	remaining: 3.78s
400:	test: 0.8569385	best: 0.8569385 (400)	total: 2.08s	remaining: 3.11s
500:	test: 0.8576410	best: 0.8576410 (500)	total: 2.58s	remaining: 2.57s
600:	test: 0.8585538	best: 0.8585538 (600)	total: 3.07s	remaining: 2.04s
700:	test: 0.8590923	best: 0.8591487 (694)	total: 3.59s	remaining: 1.53s
800:	test: 0.8593590	best: 0.8595564 (749)	total: 4.06s	remaining: 1.01s
900:	test: 0.8593282	best: 0.8595564 (749)	total: 4.55s	remaining: 500ms
999:	test: 0.8593897	best: 0.8595974 (986)	total: 5.04s	remaining: 0us

bestTest = 0.8595974359
bestIteration = 986

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	test: 0.7811641	best

In [69]:
result['params'] # лучшие параметры

{'depth': 5, 'learning_rate': 0.1}

In [72]:
result.keys() # ключи словаря

dict_keys(['params', 'cv_results'])

In [74]:
pd.DataFrame(result['cv_results'])

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.794033,0.007525,0.621375,0.000656,0.620684,0.000356
1,1,0.821947,0.006043,0.570002,0.000726,0.568969,0.001325
2,2,0.826858,0.007129,0.522675,0.000119,0.521057,0.001166
3,3,0.829708,0.005292,0.488722,0.002229,0.486886,0.003132
4,4,0.830411,0.004700,0.465270,0.004627,0.463165,0.005160
...,...,...,...,...,...,...,...
995,995,0.847281,0.001760,0.373010,0.007753,0.107246,0.002632
996,996,0.847242,0.001793,0.373098,0.007737,0.107100,0.002649
997,997,0.847240,0.001734,0.373108,0.007770,0.106980,0.002636
998,998,0.847244,0.001777,0.373106,0.007897,0.106844,0.002673


In [75]:
pd.DataFrame(result['cv_results'])['test-AUC-mean'].max()


0.8689660773549619

# Sklearn princips
https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d



    особо не используют, но знать надо. так как там есть не только модели, но методы как обрабатывать данные
    
    Scikit-learn (sklearn) широко используется для задач машинного обучения, особенно в начальных и средних этапах обучения и разработки.

    Он предоставляет не только модели для классификации, регрессии, кластеризации и т.д., но и множество инструментов для обработки данных — таких как нормализация, масштабирование, разбиение на тренировочные и тестовые данные, кросс-валидация и другие.

# Преодбработка и фичеинжениринг
https://catboost.ai/en/docs/concepts/quantization

https://www.kaggle.com/learn/feature-engineering


    для кетбуста предобратока не нужна. не нужно масштабировать данные. 

    для кетбуста есть другая предобработка, называется квантизация, разбивка на диапазоны, будет легче обучаться. 

    фичинжениринг, означает создание новых фичей на основе имеющхся. например поделить одну фичи на другую и получить новую
    например есть две фичи (возраст и зарплата) мы их делим и получаем третью фичю

    часто генерируем большое количество фичей, и смотрим что получается

# Калибровка
https://www.kaggle.com/residentmario/notes-on-classification-probability-calibration

получаем скоры. они могут соответсвоить вероятности а могут нет. И вот если они не соответсвуют вероятности, делаем калибровку.

# Catboost, тексты и эмбеддинги

https://github.com/catboost/catboost/blob/master/catboost/tutorials/events/2020_06_04_catboost_tutorial/text_features.ipynb

https://youtu.be/ZaP5qFSIcIw?t=3802


в кетбуст есть встроенная обработка текстов. если фича имеет текст, то можно прям так скормить  кетбусту. И лучше подключить видеокарту в этом случае.

в кетбусте есть возможность обработки ембедингов


# Главные навыки для аналитика
    SQL

    pandas

    Основы статистики (AB тесты)

# Главные навыки для Дата Саентиста
    Все то же, что и у аналитика

    Машинное обучение (sklearn, catboost)

    более продвинутый python (основы ООП)
    
    linux, git, docker, веб-сервисы